In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans

a) Wczytać obraz 'spring.png' (np. za pomocą biblioteki PIL) i potraktować każdy piksel jako pojedynczy punkt o 3 wymiarach.

In [ ]:
img = Image.open('spring.png')
img

In [ ]:
# przetworzenie zdjęcia na dane akceptowalne przez kolejne metody
X = np.array(img.getdata())

b) (3 pkt) Dokonać klasteryzacji metodą k-średnich i wybrać do ostatecznego podziału jako liczbę k najmniejszą z liczb, dla której wartość inercji jest mniejsza niż $ 3 \cdot 10^{8} $.

In [ ]:
i=2
inertia_list=[]
while True:
    print(f'processing for {i} clusters...')
    kmeans=KMeans(n_clusters=i, init='k-means++', random_state=1, max_iter=100).fit(X)
    inertia_list.append(kmeans.inertia_)
    if kmeans.inertia_ < 3 * 10** 8:
        print(f'{i} clusters meets the assumption inertia < 3 * 10** 8')
        break
    i+=1

c) (1 pkt) Współrzędne wszystkich uzyskanych centroidów zaokrąglić do najbliższej liczby całkowitej.

In [ ]:
kmeans.cluster_centers_ = np.around(kmeans.cluster_centers_)
kmeans.cluster_centers_

d) (1 pkt) Dla grupowania na tak ustalone k klastrów dokonać przypisania każdemu pikselowi jego zaokrąglonej wartości środka ciężkości.

In [ ]:
# etykiety przynależności do grupy
kmeans.labels_

# kolory, centra 
kmeans.cluster_centers_

# podmiana etykiet na odpowiednie wartości
pixels_to_img = np.array([kmeans.cluster_centers_[pixel] 
                            for pixel in kmeans.labels_])

e) (1 pkt) Tak utworzony obraz zapisać w pliku .png i dołączyć go do rozwiązania.

In [ ]:
# wczytanie obrazu z tablicy po odpowiednim jej przekształceniu
img_kmeans = Image.fromarray(np.uint8(
    pixels_to_img.reshape(*img.size[::-1], -1)
    ))

# zapis do pliku
img_kmeans.save('spring_compressed.png')
img_kmeans

f) (2 pkt) Przy pomocy analizy składowych głównych (PCA) dokonać redukcji wymiarowości oryginalnego obrazu 'spring.png' do 2 najważniejszych składowych głównych. Wypisać łączny procent wyjaśnionej przez nie wariancji.

In [ ]:
from sklearn.decomposition import PCA

# przygotowanie PCA do redukcji
pca = PCA(n_components=2)

# redukacja
pca.fit(X)

print(f"redukcja z 3 do 2 zmiennych zapewniła dokładność na poziomie {sum(pca.explained_variance_ratio_* 100.0):.1f}%.")

g) (2 pkt) Wykonać wykres punktowy obrazu po transformacji PCA na 2 składowe. Kolor danego punktu na wykresie ma odpowiadać jego przyporządkowaniu przez wcześniej wykonane grupowanie metodą k-średnich. Utworzyć legendę, która będzie przypisywała danemu kolorowi z wykresu kolor z przyporządkowania metodą k-średnich (np. kolor niebieski to klaster 1 z metody k-średnich, kolor zielony to klaster 2 z metody k-średnich, itd.). Do legendy użyć mapy kolorów, która pozwoli na ich czytelne odróżnienie, np. ‘Paired’, tj. dodać argument cmap=plt.get_cmap(‘Paired’). Legenda nie ma zasłaniać wyświetlanych punktów.

In [ ]:
plt.figure(figsize=(10, 7.5))
plt.title('PCA: components = 2')
plt.xlabel('com 1')
plt.ylabel('com 2')

# wyciągnięcie danych 
cent_s = kmeans.cluster_centers_
lab_s = kmeans.labels_
pca_img = pca.fit_transform(X)

# utworzenie color map
cmap = plt.get_cmap('inferno')
colors = [cmap(i/kmeans.n_clusters) 
        for i in range(kmeans.n_clusters)]


for i, cent in enumerate(cent_s):
    # wybór indexów z itą etykietą
    index = np.where(lab_s == i)
    # wykres punktowy dla i'tej etykiety
    plt.scatter(pca_img[index, 0], pca_img[index, 1],
    color = colors[i], label=f'nr {i+1}',
    marker='.')

plt.legend(title='clusters')
plt.show()